In [ ]:
import matplotlib.pyplot as plt
import DMK_go_coude as Fns
import numpy as np
import os, readcol
import scipy.optimize as optim
import scipy.interpolate as interp
import pickle

from astropy.io import fits 
from mpfit import mpfit
from scipy import signal

%matplotlib inline

In [ ]:
dir = os.getenv("HOME") + '/Research/YMG/coude_data/20140321/'
rdir = dir + 'reduction/'
codedir = os.getenv("HOME") + '/codes/coudereduction/'
#codedir = os.getenv("HOME") + '/Research/Codes/coudereduction/'

In [ ]:
os.chdir(dir)

In [ ]:
DarkCurVal = 0.0

InfoFile = 'headstrip.csv'
FileInfo = readcol.readcol( InfoFile, fsep = ',', asRecArray = True )
DarkCube = FileInfo.ExpTime * DarkCurVal

In [ ]:
BiasInds = np.where( FileInfo.Type == 'zero' )[0]
FlatInds = np.where( FileInfo.Type == 'flat' )[0]
ArcInds  = np.where( (FileInfo.Type == 'comp') & ( (FileInfo.Object == 'Thar') | (FileInfo.Object == 'THAR') | (FileInfo.Object == 'A') ) )[0]
ObjInds  = np.where( (FileInfo.Type == 'object') & (FileInfo.Object != 'SolPort') & (FileInfo.Object != 'solar port') & (FileInfo.Object != 'solar_ort') )[0]

In [ ]:
CalsDone = True
SuperBias, FlatField = Fns.Basic_Cals( FileInfo.File[BiasInds], FileInfo.File[FlatInds], CalsDone, rdir, plots = False )

In [ ]:
ShowBPM = False
BPM = Fns.Make_BPM( SuperBias, FlatField, 99.9, ShowBPM )

In [ ]:
RdNoise  = FileInfo.rdn[ArcInds] / FileInfo.gain[ArcInds]
DarkCur  = DarkCube[ArcInds] / FileInfo.gain[ArcInds]
ArcCube, ArcSNR = Fns.Make_Cube( FileInfo.File[ArcInds], RdNoise, DarkCur, Bias = SuperBias )

RdNoise  = FileInfo.rdn[ObjInds] / FileInfo.gain[ObjInds]
DarkCur  = DarkCube[ObjInds] / FileInfo.gain[ObjInds]
ObjCube, ObjSNR = Fns.Make_Cube( FileInfo.File[ObjInds], RdNoise, DarkCur, Bias = SuperBias, Flat = FlatField, BPM = BPM )

In [ ]:
OrderStart = -32
TraceDone = True
MedCut = 95.0
MedTrace, FitTrace = Fns.Get_Trace( FlatField, ObjCube, OrderStart, MedCut, rdir, TraceDone, plots = False )

In [ ]:
#fullspec, fullsig_spec = Fns.extractor( ObjCube, ObjSNR, FitTrace, quick = False, nosub = False, arc = False )

In [ ]:
spec     = pickle.load(open(rdir+'extracted_spec_oldway.pkl','rb'))
sig_spec = pickle.load(open(rdir+'extracted_sigspec_oldway.pkl','rb'))
wspec     = pickle.load(open(rdir+'extracted_wspec_oldway.pkl','rb'))
sig_wspec = pickle.load(open(rdir+'extracted_sigwspec_oldway.pkl','rb'))

In [ ]:
wspec      = wspec[:,::-1,:]
sig_wspec  = sig_wspec[:,::-1,:]
spec       = spec[:,::-1,:]
sig_spec   = sig_spec[:,::-1,:]

In [ ]:
def Gaussian( x, A, mean, sigma, const ):
    gauss = A * np.exp( - ( x - mean ) ** 2.0 / ( 2.0 * sigma ** 2 ) ) + const
    return gauss
    
def TwoGaussian( x, A1, mean1, sigma1, const, A2, mean2, sigma2 ):
    return Gaussian( x, A1, mean1, sigma1, const ) + Gaussian( x, A2, mean2, sigma2, const )

In [ ]:
THAR      = fits.open( codedir + 'thar_photron.fits' )[0]
THARhead  = THAR.header
THARspec  = THAR.data
THARwav   = np.arange( len( THARspec ) ) * THARhead['CDELT1'] + THARhead['CRVAL1']
THARlines = readcol.readcol( codedir + 'ThAr_list.txt', asRecArray = True )

logTHARspec = np.log10(THARspec)

#qwe            = np.where((THARwav >= np.min(roughwav-10)) & (THARwav <= np.max(roughwav+10)))[0] 
#ls             = THARspec[qwe] - np.min(THARspec[qwe])
#region         = np.where(ls < np.median(ls))
#ls[region]     = np.median(ls)

In [ ]:
roughdisp  = 7000.0 / 60000.0 / 2.0
wavestart  = 6957.0
startorder = wspec.shape[1] - 15 - 1
arcspec    = wspec[0,startorder,:]
roughwav   = wavestart + roughdisp * np.arange( len( arcspec ) )

arcspec           = arcspec - np.min( arcspec )
belowmed          = np.where( arcspec < np.median( arcspec ) )
arcspec[belowmed] = np.median( arcspec )
logarcspec        = np.log10( arcspec )
logarcspec        = logarcspec - np.min( logarcspec )

In [ ]:
peaks = signal.find_peaks_cwt( arcspec, np.arange( 1, 2 ), min_snr = 1000, noise_perc = 20 )

plt.plot( logarcspec, 'k-' )
for peak in peaks:
    plt.axvline( x = peak, color = 'r', ls = ':' )
plt.xlim( 300, 500 )
plt.show()

In [ ]:
def Find_Peaks_Wavelet( wav, spec, peaksnr = 5, pwidth = 10, minsep = 1 ):
    
    # Find peaks using the cwt routine from scipy.signal
    peaks = signal.find_peaks_cwt( arcspec, np.arange( 1, 2 ), min_snr = peaksnr, noise_perc = 20 )
    
    # Offset from start/end of spectrum by some number of pixels
    peaks = peaks[ (peaks > pwidth) & (peaks < len(spec) - pwidth) ]
    
    pixcent = np.array([])
    wavcent = np.array([])
        
    for peak in peaks:
        
        xi   = wav[peak - pwidth:peak + pwidth]
        yi   = spec[peak - pwidth:peak + pwidth]
        inds = np.arange( len(xi), dtype = float )
        
        pguess   = [ yi[9], np.median( inds ), 0.9, np.median( spec ) ]
        lowerbds = [ 0.1*pguess[0], pguess[1] - 2.0, 0.3, 0.0  ]
        upperbds = [ np.inf, pguess[1] + 2.0, 1.5, np.inf ]
        
        try:
            params, pcov = optim.curve_fit( Gaussian, inds, yi, p0 = pguess, bounds = (lowerbds,upperbds) )
            
            pixval  = peak - pwidth + params[1]
            pixcent = np.append( pixcent, pixval )
            
            ceiling = np.ceil( pixval ).astype(int)
            floor   = np.floor( pixval ).astype(int)
            slope   = ( wav[ceiling] - wav[floor] ) / ( ceiling - floor )
            wavval  = wav[floor] + slope * ( pixval - floor )
            wavcent = np.append( wavcent, wavval )
            
        except RuntimeError:
            pixval  = 'nan'
            
    vals = spec[pixcent.astype(int)]
    oks  = np.ones( len(pixcent), int )
    
    for i in range( len(wavcent) ):
        dist  = np.absolute( wavcent - wavcent[i] )
        close = np.where( dist <= minsep )[0]
        small = np.where( vals[close] < np.max( vals[close] ) )[0]
        if len(small) != 0: oks[close[small]] = -1
    keep    = np.where( oks == 1 )
    pixcent = pixcent[keep]
    wavcent = wavcent[keep]
            
    return pixcent, wavcent

In [ ]:
# Run 1 -- Min SNR = 1000
snr = 1000
pixcent1, wavcent1 = Find_Peaks_Wavelet( roughwav, arcspec, peaksnr = snr )

plt.plot( logarcspec, 'k-' )
for peak in pixcent1:
    plt.axvline( peak, color = 'r', ls = ':' )
plt.show()

THARthisord = [ line for line in THARlines.wav if line > roughwav[0] and line < roughwav[-1] ]

matchedpix  = np.array([])
matchedwav  = np.array([])
matchedline = np.array([])

for i in range( len( wavcent1 ) ):
    dists      = np.absolute( THARthisord - wavcent1[i] )
    mindist    = np.min( dists )
    mindistind = np.argmin( dists )
    if mindist <= 1.5:
        matchedpix  = np.append( matchedpix, pixcent1[i] )
        matchedwav  = np.append( matchedwav, wavcent1[i] )
        matchedline = np.append( matchedline, THARthisord[mindistind] )
    
wavparams1 = np.polyfit( matchedpix, matchedline, 3 )
wavsol1    = np.polyval( wavparams1, np.arange( len( arcspec ) ) )
newwavs1   = np.polyval( wavparams1, matchedpix )

plt.plot( newwavs1, newwavs1 - matchedwav, 'kx' )
plt.xlim( min(wavsol1) - 5, max(wavsol1) + 5 )
plt.show()

plt.plot( THARwav, logTHARspec, 'r-' )
plt.plot( roughwav, logarcspec, 'b:' )
plt.plot( wavsol1, logarcspec, 'k-' )
plt.xlim( min(wavsol1) - 5, max(wavsol1) + 5 )
plt.show()

In [ ]:
# Run 2 -- Min SNR = 500
snr = 500
pixcent2, wavcent2 = Find_Peaks_Wavelet( wavsol1, arcspec, peaksnr = snr )

plt.plot( logarcspec, 'k-' )
for peak in pixcent2:
    plt.axvline( peak, color = 'r', ls = ':' )
plt.show()

THARthisord = [ line for line in THARlines.wav if line > wavsol1[0] and line < wavsol1[-1] ]

matchedpix  = np.array([])
matchedwav  = np.array([])
matchedline = np.array([])

for i in range( len( wavcent2 ) ):
    dists      = np.absolute( THARthisord - wavcent2[i] )
    mindist    = np.min( dists )
    mindistind = np.argmin( dists )
    if mindist <= 1.5:
        matchedpix  = np.append( matchedpix, pixcent2[i] )
        matchedwav  = np.append( matchedwav, wavcent2[i] )
        matchedline = np.append( matchedline, THARthisord[mindistind] )
    
wavparams2 = np.polyfit( matchedpix, matchedline, 5 )
wavsol2    = np.polyval( wavparams2, np.arange( len( arcspec ) ) )
newwavs2   = np.polyval( wavparams2, matchedpix )

plt.plot( newwavs2, newwavs2 - matchedwav, 'kx' )
plt.xlim( min(wavsol2) - 5, max(wavsol2) + 5 )
plt.show()

plt.plot( THARwav, logTHARspec, 'r-' )
plt.plot( wavsol1, logarcspec, 'b:' )
plt.plot( wavsol2, logarcspec, 'k-' )
plt.xlim( min(wavsol2) - 5, max(wavsol2) + 5 )
plt.show()

In [ ]:
# Run 3 -- Min SNR = 100
snr = 100
pixcent3, wavcent3 = Find_Peaks_Wavelet( wavsol2, arcspec, peaksnr = snr )

plt.plot( logarcspec, 'k-' )
for peak in pixcent3:
    plt.axvline( peak, color = 'r', ls = ':' )
plt.show()

THARthisord = [ line for line in THARlines.wav if line > wavsol2[0] and line < wavsol2[-1] ]

matchedpix  = np.array([])
matchedwav  = np.array([])
matchedline = np.array([])

for i in range( len( wavcent3 ) ):
    dists      = np.absolute( THARthisord - wavcent3[i] )
    mindist    = np.min( dists )
    mindistind = np.argmin( dists )
    if mindist <= 1.5:
        matchedpix  = np.append( matchedpix, pixcent3[i] )
        matchedwav  = np.append( matchedwav, wavcent3[i] )
        matchedline = np.append( matchedline, THARthisord[mindistind] )
    
wavparams3 = np.polyfit( matchedpix, matchedline, 5 )
wavsol3    = np.polyval( wavparams3, np.arange( len( arcspec ) ) )
newwavs3   = np.polyval( wavparams3, matchedpix )

plt.plot( newwavs3, newwavs3 - matchedwav, 'kx' )
plt.xlim( min(wavsol3) - 5, max(wavsol3) + 5 )
plt.show()

plt.plot( THARwav, logTHARspec, 'r-' )
plt.plot( wavsol2, logarcspec, 'b:' )
plt.plot( wavsol3, logarcspec, 'k-' )
plt.xlim( min(wavsol3) - 5, max(wavsol3) + 5 )
plt.show()

In [ ]:
# Run 4 -- Min SNR = 20
snr = 20
pixcent4, wavcent4 = Find_Peaks_Wavelet( wavsol3, arcspec, peaksnr = snr )

plt.plot( logarcspec, 'k-' )
for peak in pixcent4:
    plt.axvline( peak, color = 'r', ls = ':' )
plt.show()

THARthisord = [ line for line in THARlines.wav if line > wavsol3[0] and line < wavsol3[-1] ]

matchedpix  = np.array([])
matchedwav  = np.array([])
matchedline = np.array([])

for i in range( len( wavcent4 ) ):
    dists      = np.absolute( THARthisord - wavcent4[i] )
    mindist    = np.min( dists )
    mindistind = np.argmin( dists )
    if mindist <= 1.5:
        matchedpix  = np.append( matchedpix, pixcent4[i] )
        matchedwav  = np.append( matchedwav, wavcent4[i] )
        matchedline = np.append( matchedline, THARthisord[mindistind] )
    
wavparams4 = np.polyfit( matchedpix, matchedline, 6 )
wavsol4    = np.polyval( wavparams4, np.arange( len( arcspec ) ) )
newwavs4   = np.polyval( wavparams4, matchedpix )

plt.plot( newwavs4, newwavs4 - matchedwav, 'kx' )
plt.xlim( min(wavsol4) - 5, max(wavsol4) + 5 )
plt.show()

plt.plot( THARwav, logTHARspec, 'r-' )
plt.plot( roughwav, logarcspec, 'b:' )
plt.plot( wavsol4, logarcspec, 'k-' )
plt.xlim( min(wavsol4) - 5, max(wavsol4) + 5 )
plt.show()

In [ ]:
# Run 4 -- Min SNR = 20
snr = 5
pixcent5, wavcent5 = Find_Peaks_Wavelet( wavsol4, arcspec, peaksnr = snr )

plt.plot( logarcspec, 'k-' )
for peak in pixcent5:
    plt.axvline( peak, color = 'r', ls = ':' )
plt.show()

THARthisord = [ line for line in THARlines.wav if line > wavsol4[0] and line < wavsol4[-1] ]

matchedpix  = np.array([])
matchedwav  = np.array([])
matchedline = np.array([])

for i in range( len( wavcent5 ) ):
    dists      = np.absolute( THARthisord - wavcent5[i] )
    mindist    = np.min( dists )
    mindistind = np.argmin( dists )
    if mindist <= 1.5:
        matchedpix  = np.append( matchedpix, pixcent5[i] )
        matchedwav  = np.append( matchedwav, wavcent5[i] )
        matchedline = np.append( matchedline, THARthisord[mindistind] )
    
wavparams5 = np.polyfit( matchedpix, matchedline, 6 )
wavsol5    = np.polyval( wavparams5, np.arange( len( arcspec ) ) )
newwavs5   = np.polyval( wavparams5, matchedpix )

plt.plot( newwavs5, newwavs5 - matchedwav, 'kx' )
plt.xlim( min(wavsol5) - 5, max(wavsol5) + 5 )
plt.show()

plt.plot( THARwav, logTHARspec, 'r-' )
plt.plot( roughwav, logarcspec, 'b:' )
plt.plot( wavsol5, logarcspec, 'k-' )
plt.xlim( 7010, 7040 )
plt.show()

In [ ]:
pixcent, wavcent = Find_Peaks_Wavelet( roughwav, arcspec, peaksnr=5 )
print wavcent
print THARthisord

In [ ]:
THARthisord = [ line for line in THARlines.wav if line > roughwav[0] and line < roughwav[-1] ]

matchedpix  = np.array([])
matchedwav  = np.array([])
matchedline = np.array([])

for i in range( len( wavcent ) ):
    dists      = np.absolute( THARthisord - wavcent[i] )
    mindist    = np.min( dists )
    mindistind = np.argmin( dists )
    if mindist <= 1.5:
        matchedpix  = np.append( matchedpix, pixcent[i] )
        matchedwav  = np.append( matchedwav, wavcent[i] )
        matchedline = np.append( matchedline, THARthisord[mindistind] )
    
wavparams = np.polyfit( matchedpix, matchedline, 5 )
wavspline = interp.InterpolatedUnivariateSpline( matchedpix, matchedline )
wavsol    = np.polyval( wavparams, np.arange( len( arcspec ) ) )
wavsolspl = wavspline( np.arange( len( arcspec ) ) )

newwavs    = np.polyval( wavparams, matchedpix )
newwavsspl = wavspline( matchedpix )

#plt.plot( matchedline, newwavs - matchedline, 'ko' )
plt.plot( matchedline, newwavsspl - matchedline, 'rx' )
plt.show()

#print matchedwav
#print matchedline

plt.plot( wavsolspl, logarcspec, 'k-' )
plt.plot( THARwav, logTHARspec, 'r-' )
plt.xlim( 6950, 7000 )
plt.show()

In [ ]:

#THARthisord = [ line for line in THARlines.wav if line > thisorder[0] and line < thisorder[1] ]
#
#matched = np.zeros( len( wavcent ) )
#for i in range( len( wavcent ) ):
#    dists      = np.absolute( THARlines.wav - wavcent[i] )
#    mindist    = np.argmin( dists )
#    line       = THARlines.wav[mindist]
#    matched[i] = line
#    
#wavparams1 = np.polyfit( pixcent, matched, 3 )
#wavsol1    = np.polyval( wavparams1, np.arange( len( arcspec ) ) )
#
#newwavcent = np.polyval( wavparams1, pixcent )
#plt.plot( matched, newwavcent - matched, 'ko' )
#plt.show()

In [ ]:
#linetest = 6
#testwavs = [ wav for wav in wavsol1 if wav >= THARthisord[linetest] - 2 and wav <= THARthisord[linetest] + 2 ]
#testinds = [ np.where( wavsol1 == wav )[0][0] for wav in testwavs ]
#
#pguess = [ np.max(arcspec[testinds]), testwavs[np.argmax(arcspec[testinds])], 0.1, 0 ]
#params, pcov = optim.curve_fit( Gaussian, testwavs, arcspec[testinds], p0 = pguess )
#fit = Gaussian( testwavs, params[0], params[1], params[2], params[3] )
#
#plt.plot( testwavs, arcspec[testinds], 'k-' )
#plt.plot( testwavs, Gaussian(testwavs, params[0], params[1], params[2], params[3] ), 'r.' )
#plt.show()

In [ ]:
#fitsets = np.split( THARthisord, np.where( np.diff( THARthisord ) > 1 )[0]+1)
#linetest = fitsets[14]
#
#testwavs = [ wav for wav in wavsol1 if np.median(linetest) - 2 <= wav <= np.median(linetest) + 2 ]
#testinds = [ np.where( wavsol1 == wav )[0][0] for wav in testwavs ]
#
#pguess = [ np.max( arcspec[testinds] ), testwavs[np.argmax(arcspec[testinds])], 0.1, 0 ]
#params, pcov = optim.curve_fit( Gaussian, testwavs, arcspec[testinds], p0 = pguess )
#fit = Gaussian( testwavs, params[0], params[1], params[2], params[3] )
#
#pguess = [ np.max( arcspec[testinds] ), linetest[0], 0.1, 0, np.max( arcspec[testinds] ), linetest[1], 0.1 ]
#params, pcov = optim.curve_fit( TwoGaussian, testwavs, arcspec[testinds], p0 = pguess )
#fit = TwoGaussian( testwavs, params[0], params[1], params[2], params[3], params[4], params[5], params[6] )
#print params[5]
#
#plt.plot( testwavs, arcspec[testinds], 'k-' )
#plt.plot( testwavs, fit, 'r-' )
#plt.show()